## 3.1 Dardanel Lojistik Depo Optimizasyonu (81 İl)

Dardanel firması, Türkiye genelinde ürünlerini soğuk zincir bozulmadan dağıtmak amacıyla **Çanakkale merkezli bir lojistik ağı** planlamak istemektedir. Amaç, Türkiye’deki 81 il için **en uygun 10 ilin depo** olarak seçilmesi ve bu depolardan diğer illere ürünlerin dağıtılmasıdır.

### Problem Tanımı

- Tüm iller arası mesafe matrisi mevcuttur (ilmesafe.xlsx).
- Her il yalnızca bir depoya bağlanmalıdır.
- Depolar arası mesafeler göz önünde bulundurularak, toplam lojistik maliyet (mesafeye dayalı) **minimize edilmelidir**.
- **Çanakkale sabit depo** olarak seçilmiştir.
- Geriye kalan **9 depo** optimizasyonla seçilecektir.

### QUBO Modeli

Her il $ i $ ve her potansiyel depo $ j $ için bir ikili değişken tanımlanır:

- $ x_{ij} = 1 $ ise, il $ i $, depo $ j $'ye atanmıştır.
- Amaç fonksiyonu:
  $$
  \min \sum_{i=1}^{n} \sum_{j=1}^{k} d_{ij} \cdot x_{ij}
  $$
  Burada $ d_{ij} $ Çanakkale ile diğer iller arası mesafedir (veya seçilen depoyla).

- Kısıtlar:
  - Her il yalnızca bir depoya bağlanmalı: 
    $$
    \sum_{j=1}^{k} x_{ij} = 1, \quad \forall i
    $$
  - Seçilen depo sayısı: 
    $$
    \sum_{j=1}^{n} y_j = 10
    $$

QUBO dönüşümü, bu kısıtların ceza terimleri ile modele dahil edilmesiyle yapılır.

---

## 3.1.1 İl-Depo Ataması ve Harita Görselleştirmesi

Optimizasyon tamamlandıktan sonra elde edilen:

- 📦 **Seçilen Depo İlleri** listesi,
- 🏙️ **Her ilin bağlı olduğu depo** bilgisi,

interaktif bir harita üzerinde görselleştirilmiştir.

Bu görselleştirme için kullanılan araçlar:

- `folium`: Türkiye haritası üzerinde il merkezlerine dairesel işaretleyici (marker) koymak
- `pandas`: Atama tablosunu hazırlamak
- `networkx` veya `plotly`: Gelişmiş bağlantı haritaları

### Örnek Görsel:

- Depolar kırmızı marker ile gösterilmiştir.
- Diğer iller bağlı oldukları depoya yönlendirilmiştir.
- Kullanıcı bu görselden lojistik bağlantıların yapısını analiz edebilir.

Bu yapı ile sadece maliyet değil, **operasyonel verimlilik** ve **coğrafi kapsama** da analiz edilebilir hale gelmiştir.


In [1]:
import pandas as pd
from dimod import BinaryQuadraticModel
from itertools import combinations
from neal import SimulatedAnnealingSampler
from collections import defaultdict

# 1. Veriyi yükle
df = pd.read_excel("ilmesafe.xlsx", header=1)
df.rename(columns={df.columns[0]: "Plaka", df.columns[1]: "İL ADI"}, inplace=True)
cities = df["İL ADI"].tolist()

# 2. Mesafeleri oku
distances = {}
for i, row in df.iterrows():
    from_city = row["İL ADI"]
    for to_city in df.columns[2:]:
        dist = row[to_city]
        if pd.notna(dist):
            distances[(from_city, to_city)] = dist
            distances[(to_city, from_city)] = dist

# 3. Sabit seçilen depolar (örnek: optimizasyonla seçilmiş ya da kullanıcı tarafından belirlenmiş)
fixed_depots = [
    "ÇANAKKALE", "MERSİN", "KAYSERİ", "İSTANBUL", "SAMSUN",
    "BİTLİS", "ANTALYA", "KOCAELİ (İZMİT)", "GAZİANTEP", "ERZURUM"
]

# 4. BQM modeli
bqm = BinaryQuadraticModel("BINARY")

# 5. y_ij değişkenleri: her şehir sadece sabit 10 depodan birine atanabilir
A = 1000  # ceza katsayısı
B = 1     # taşıma maliyeti katsayısı

for j in cities:
    if j in fixed_depots:
        continue  # depo kendine atanmaz
    assign_vars = []
    for i in fixed_depots:
        if i != j:
            yij = f"y_{i}_{j}"
            d = distances.get((i, j), None)
            if d:
                bqm.add_variable(yij, B * d)
                assign_vars.append(yij)
    
    # Atama kısıtı: her şehir sadece 1 depoya atanmalı → (∑ y_ij - 1)^2
    for var in assign_vars:
        bqm.add_variable(var, bqm.get_linear(var) + A * (1 - 2))
    for u, v in combinations(assign_vars, 2):
        bqm.add_interaction(u, v, 2 * A)
    bqm.offset += A

# 6. Simulated Annealing ile çözüm
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads=100)
best = sampleset.first.sample

# 7. Atama çıktıları
assignment = defaultdict(list)
for j in cities:
    if j in fixed_depots:
        continue
    for i in fixed_depots:
        if i != j and best.get(f"y_{i}_{j}", 0) == 1:
            assignment[i].append(j)

# 8. Çıktılar
print(f"📦 Seçilen Depolar ({len(fixed_depots)} toplam):")
for depo in fixed_depots:
    print(f"✅ {depo}")

print("\n🏙️ İl Atamaları:")
for depo in fixed_depots:
    print(f"\n### ✅ {depo}")
    if assignment[depo]:
        for il in sorted(assignment[depo]):
            print(f"- {il}")
    else:
        print("- (Bağlı il yok)")

📦 Seçilen Depolar (10 toplam):
✅ ÇANAKKALE
✅ MERSİN
✅ KAYSERİ
✅ İSTANBUL
✅ SAMSUN
✅ BİTLİS
✅ ANTALYA
✅ KOCAELİ (İZMİT)
✅ GAZİANTEP
✅ ERZURUM

🏙️ İl Atamaları:

### ✅ ÇANAKKALE
- BALIKESİR
- EDİRNE
- MANİSA
- İZMİR

### ✅ MERSİN
- ADANA
- HATAY
- KARAMAN

### ✅ KAYSERİ
- AKSARAY
- ANKARA
- KIRIKKALE
- KIRŞEHİR
- KONYA
- NEVŞEHİR
- NİĞDE
- SİVAS
- TOKAT
- YOZGAT
- ÇORUM

### ✅ İSTANBUL
- KIRKLARELİ
- TEKİRDAĞ

### ✅ SAMSUN
- AMASYA
- GİRESUN
- KASTAMONU
- ORDU
- SİNOP
- ÇANKIRI

### ✅ BİTLİS
- BATMAN
- BİNGÖL
- DİYARBAKIR
- ELAZIĞ
- HAKKARİ
- MARDİN
- MUŞ
- SİİRT
- VAN
- ŞIRNAK

### ✅ ANTALYA
- AFYONKARAHİSAR
- AYDIN
- BURDUR
- DENİZLİ
- ISPARTA
- MUĞLA
- UŞAK

### ✅ KOCAELİ (İZMİT)
- BARTIN
- BOLU
- BURSA
- BİLECİK
- DÜZCE
- ESKİŞEHİR
- KARABÜK
- KÜTAHYA
- SAKARYA (ADAPAZARI)
- YALOVA
- ZONGULDAK

### ✅ GAZİANTEP
- ADIYAMAN
- KAHRAMANMARAŞ
- KİLİS
- MALATYA
- OSMANİYE
- ŞANLIURFA

### ✅ ERZURUM
- ARDAHAN
- ARTVİN
- AĞRI
- BAYBURT
- ERZİNCAN
- GÜMÜŞHANE
- IĞDIR
- KARS
- RİZE
- TRABZON
- 

In [2]:
import pandas as pd
import folium

# Dosyaları oku
coords_df = pd.read_csv("turkiye_il_koordinatlari.csv")
assignments_df = pd.read_csv("il_depo_atamalariv1.csv")

# Sabit 10 depo
fixed_depots = [
    "ÇANAKKALE", "MERSİN", "KAYSERİ", "İSTANBUL", "SAMSUN",
    "BİTLİS", "ANTALYA", "KOCAELİ (İZMİT)", "GAZİANTEP", "ERZURUM"
]

# Haritayı oluştur
m = folium.Map(location=[39.0, 35.5], zoom_start=6)

# Her depo için kırmızı marker
for depo in fixed_depots:
    row = coords_df[coords_df["İL"] == depo]
    if not row.empty:
        folium.CircleMarker(
            location=[row["LAT"].values[0], row["LON"].values[0]],
            radius=7,
            color="red",
            fill=True,
            fill_opacity=0.9,
            popup=f"Depo: {depo}"
        ).add_to(m)

# Tüm iller için bağlantı çiz ve marker ekle
for _, row in assignments_df.iterrows():
    il = row["İL"]
    depo = row["DEPO"]
    
    il_coord = coords_df[coords_df["İL"] == il]
    depo_coord = coords_df[coords_df["İL"] == depo]
    
    if not il_coord.empty and not depo_coord.empty:
        il_lat, il_lon = il_coord["LAT"].values[0], il_coord["LON"].values[0]
        depo_lat, depo_lon = depo_coord["LAT"].values[0], depo_coord["LON"].values[0]
        
        # İl markerı
        folium.CircleMarker(
            location=[il_lat, il_lon],
            radius=4,
            color="blue",
            fill=True,
            fill_opacity=0.6,
            popup=il
        ).add_to(m)
        
        # Bağlantı çizgisi
        folium.PolyLine(
            locations=[[il_lat, il_lon], [depo_lat, depo_lon]],
            color="gray", weight=1.5, opacity=0.5
        ).add_to(m)

# Haritayı HTML dosyası olarak kaydet
m.save("depo_atama_haritasi.html")
print("✅ Harita oluşturuldu: depo_atama_haritasi.html")


✅ Harita oluşturuldu: depo_atama_haritasi.html


In [4]:
import pandas as pd
import folium
import branca.colormap as cm
from folium import plugins

# Veri dosyalarını yükle
mesafe_df = pd.read_csv("ilmesafe_long_clean_v2.csv")
coords_df = pd.read_csv("turkiye_il_koordinatlari.csv")
assignments_df = pd.read_csv("il_depo_atamalariv1.csv")

# Sabit 10 depo listesi
fixed_depots = [
    "ÇANAKKALE", "MERSİN", "KAYSERİ", "İSTANBUL", "SAMSUN",
    "BİTLİS", "ANTALYA", "KOCAELİ (İZMİT)", "GAZİANTEP", "ERZURUM"
]

# Harita oluştur
m = folium.Map(location=[39.0, 35.5], zoom_start=6, tiles="cartodbpositron")

# Renk skalası (mesafe için)
min_dist = mesafe_df["MESAFE"].min()
max_dist = mesafe_df["MESAFE"].max()
colormap = cm.LinearColormap(colors=['green', 'yellow', 'red'], vmin=min_dist, vmax=max_dist)

# Toplam mesafe hesaplama
total_distance = 0
connection_count = 0

# Depo markerları
for depo in fixed_depots:
    row = coords_df[coords_df["İL"] == depo]
    if not row.empty:
        folium.CircleMarker(
            location=[row["LAT"].values[0], row["LON"].values[0]],
            radius=7,
            color="red",
            fill=True,
            fill_opacity=0.9,
            popup=f"Depo: {depo}"
        ).add_to(m)

# Bağlantı çizgileri
for _, row in assignments_df.iterrows():
    il = row["İL"]
    depo = row["DEPO"]

    il_coord = coords_df[coords_df["İL"] == il]
    depo_coord = coords_df[coords_df["İL"] == depo]

    if il_coord.empty or depo_coord.empty:
        continue

    il_lat, il_lon = il_coord["LAT"].values[0], il_coord["LON"].values[0]
    depo_lat, depo_lon = depo_coord["LAT"].values[0], depo_coord["LON"].values[0]

    # Mesafe verisi çek
    mesafe1 = mesafe_df[(mesafe_df["İL1"] == il) & (mesafe_df["İL2"] == depo)]["MESAFE"].values
    mesafe2 = mesafe_df[(mesafe_df["İL2"] == il) & (mesafe_df["İL1"] == depo)]["MESAFE"].values
    mesafe = mesafe1[0] if len(mesafe1) > 0 else (mesafe2[0] if len(mesafe2) > 0 else None)

    if mesafe is not None:
        total_distance += mesafe
        connection_count += 1

        folium.PolyLine(
            locations=[[il_lat, il_lon], [depo_lat, depo_lon]],
            color=colormap(mesafe),
            weight=2,
            opacity=0.7,
            tooltip=f"{il} → {depo}: {mesafe} km"
        ).add_to(m)

        # İl marker
        folium.CircleMarker(
            location=[il_lat, il_lon],
            radius=3,
            color="blue",
            fill=True,
            fill_opacity=0.6,
            popup=il
        ).add_to(m)

# Lojistik analiz çıktısı
print("🔢 Toplam bağlantı sayısı:", connection_count)
print("🚚 Toplam lojistik mesafe:", round(total_distance, 2), "km")
print("📏 Ortalama mesafe:", round(total_distance / connection_count, 2), "km")

# Renk skalası haritaya ekle
colormap.caption = "Mesafe Renk Skalası (km)"
colormap.add_to(m)

# Haritayı kaydet
m.save("depo_atama_haritasi_mesafe.html")
print("✅ Harita oluşturuldu: depo_atama_haritasi_mesafe.html")


🔢 Toplam bağlantı sayısı: 79
🚚 Toplam lojistik mesafe: 42104 km
📏 Ortalama mesafe: 532.96 km
✅ Harita oluşturuldu: depo_atama_haritasi_mesafe.html
